In [19]:
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split as sample
from sklearn.manifold import MDS
# делаем импорт всех необходимых библиотек
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid", palette="muted")
# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")

## Часть 2: Cоздание новых признаков

_Над этой частью работала Халифаева Лейла и Рыбьякова Лиза_

1) В дататасетах есть информация об Индексе массы тела, поэтому мы решили создать в каждом из них новую переменную, которая будет отражать, является ли вес респондента избыточным(1) или нет(0), то есть превышает ли норму в 25 единиц или нет.

In [20]:
data_heart= pd.read_csv('data_heart.csv',index_col=0)
data_heart.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,BMI_excess,healthy_lifestyle
215201,0.0,0.0,1.0,1.0,17.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,0.0,1.0,5.0,6.0,6.0,0.0,1.0
27524,0.0,1.0,1.0,1.0,33.0,1.0,0.0,2.0,0.0,1.0,...,5.0,5.0,10.0,1.0,1.0,4.0,2.0,6.0,1.0,0.0
149754,0.0,1.0,0.0,1.0,38.0,1.0,0.0,0.0,0.0,1.0,...,3.0,5.0,0.0,0.0,0.0,4.0,5.0,5.0,1.0,0.0
1436,0.0,1.0,0.0,1.0,29.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,1.0,12.0,4.0,6.0,1.0,0.0
206341,0.0,0.0,1.0,1.0,27.0,0.0,0.0,2.0,1.0,0.0,...,2.0,0.0,2.0,0.0,0.0,9.0,5.0,5.0,1.0,0.0


In [21]:
data_stroke= pd.read_csv('data_stroke.csv',index_col=0)
data_stroke.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,ever_married_Yes,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,BMI_excess,glucose_level_normal
2411,60.0,0,0,87.86,29.0,0,1,1,1,0,0,0,1,0,0,1.0,1.0
2109,47.0,0,0,131.43,24.3,0,0,1,1,0,0,1,0,1,0,0.0,0.0
3906,19.0,0,0,56.33,29.4,0,1,0,1,0,0,0,0,0,0,1.0,0.0
1138,53.0,0,0,78.73,23.3,0,1,1,0,0,0,0,0,1,0,0.0,0.0
3714,41.0,0,0,92.14,29.6,0,0,1,1,0,0,1,1,0,0,1.0,1.0


In [22]:
data_heart['BMI_excess'] = data_heart['BMI']
data_heart['BMI_excess'] = np.where((data_heart['BMI'] < 25), 0, data_heart['BMI_excess'])
data_heart['BMI_excess'] = np.where((data_heart['BMI'] >= 25), 1, data_heart['BMI_excess'])

In [23]:
data_stroke['BMI_excess'] = data_stroke['bmi']
data_stroke['BMI_excess'] = np.where((data_stroke['bmi'] < 25), 0, data_stroke['BMI_excess'])
data_stroke['BMI_excess'] = np.where((data_stroke['bmi'] >= 25), 1, data_stroke['BMI_excess'])

2) Также в данных об инсультах есть переменная, отражающая средний уровень глюкозы в крови респондента. Создадим пременную, которая покажет, является ли показатель респондента нормальным(1) или нет(0). Нормой является уровень в пределах от 80 до 115 мг/дл.

In [24]:
data_stroke['glucose_level_normal'] = data_stroke['avg_glucose_level']
data_stroke['glucose_level_normal'] = np.where(((data_stroke['avg_glucose_level'] >= 80) & 
                                                (data_stroke['avg_glucose_level'] <= 115)), 1,
                                                 data_stroke['glucose_level_normal'])
data_stroke['glucose_level_normal'] = np.where(((data_stroke['avg_glucose_level'] < 80) | 
                                                (data_stroke['avg_glucose_level'] > 115)), 0,
                                                 data_stroke['glucose_level_normal'])

3) В первом датасете есть много признаков, которые говорят в целом о состоянии здоровья и образе жизни респондентов, поэтому можно создать еще один признак. 

_Предположим, что человек, который ведет здоровый образ жизни, соответствует следующим критериям:_
- Не курит;
- Употребляет достаточное количество фруктов и овощей в пищу;
- Регулярно является физически активным в течение месяца;
- Не употребляют алкоголь;

In [25]:
data_heart['healthy_lifestyle'] = np.zeros(data_heart.shape[0])
data_heart['healthy_lifestyle'] = np.where(((data_heart['Smoker'] == 0) &
                                            (data_heart['Fruits'] == 1) &
                                            (data_heart['Veggies'] == 1) &
                                            (data_heart['PhysActivity'] == 1) &
                                            (data_heart['HvyAlcoholConsump'] == 0)),
                                            1, data_heart['healthy_lifestyle'])